In [2]:
import ROOT as rt
# import root_numpy as rtnp
import csv
import re
import sys
import collections

from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot

import CMS_lumi, tdrstyle
a = tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0

wH = 1
Z_MASS = 91.2


# donotdelete = []
print(sys.version)

3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [3]:
fpath_bkg =OrderedDict()
tree_bkg = OrderedDict()
tree_sig = OrderedDict()
fpath_sig =OrderedDict()


start_t = time.time()



isData = 1
if isData:
    ntupler_version = 'V1p17/'
    analyzer_version = "/v5/v87/"
    analyzer_version = "/v5/v95/"
    analyzer_version = "/v5/v98/"
    analyzer_version = "/v5/v99/"
    analyzer_version = "/v5/v100/"
    analyzer_version = "/v5/v111/"
    analyzer_version = "/v5/v130/"
    analyzer_version = "/v5/v137/"
    analyzer_version = "/v5/v138/"


# #     data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/Data2018/'+analyzer_version+'/normalized/'
#     data_path = '/storage/af/user/christiw/login-1/christiw/LLP/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/Data2018/'+analyzer_version+'/normalized/'
    data_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/Data2018/'+analyzer_version+'/normalized/'

    fpath_bkg['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root'
#     fpath_bkg['data18'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2018_17Sept2018_Run2018-HighMET-17Sep2018_goodLumi.root'
#     data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/Data2017/'+analyzer_version+'/normalized/'
#     fpath_bkg['data17'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2017_Run2017-HighMET-17Nov2017_goodLumi.root'
#     data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/Data2016/'+analyzer_version+'/normalized/'
#     fpath_bkg['data16'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2016_Run2016-HighMET-07Aug17_goodLumi.root'

#     fpath_bkg['data'] = '/storage/af/user/christiw/login-1/christiw/LLP/displacedJetMuonAnalyzer/csc/V1p17//Data2018//v5/v127//normalized/Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root'


#     fpath_bkg['data17'] = '/storage/af/user/christiw/login-1/christiw/LLP/displacedJetMuonAnalyzer/csc/V1p17//Data2017//v5/v127//normalized/Run2_displacedJetMuonNtupler_V1p17_Data2017_Run2017-HighMET-17Nov2017_goodLumi.root'


else:
    ntupler_version = 'V1p17/'
    analyzer_version = "/v1/v86/"
#     analyzer_version = "/v1/v95/"
    analyzer_version = "/v1/v100/"
    analyzer_version = "/v1/v108/" #new acceptance definition

    analyzer_version = "/v1/v110/" #new acceptance definition and no cuts at all



    decay = '4Tau'
    mass = '55'
    lumi = [ 35920, 41530, 59740 ]
    years = ['MC_Summer16', 'MC_Fall17', 'MC_Fall18']
    tune = ['TuneCUETP8M1_13TeV-powheg-pythia8','TuneCP5_13TeV-powheg-pythia8','TuneCP5_13TeV-powheg-pythia8']
    ctaus  = ['100','1000','10000','100000']
    year_index = 3
    for ct in ctaus:
        
        if year_index == 3:
            mc_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_all/'+analyzer_version+'/normalized/'
            fpath_bkg['m'+mass+'ctau'+ct] = mc_path + 'ggH_HToSSTo'+decay+'_MH-125_MS-'+str(mass)+'_ctau-'+ct+'_137000pb_weighted.root'
        else:
            mc_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/'+years[year_index]+'/'+analyzer_version+'/normalized/'
            fpath_bkg['m'+mass+'ctau'+ct] = mc_path + 'ggH_HToSSTo'+decay+'_MH-125_MS-'+str(mass)+'_ctau-'+ct+'_'+tune[year_index]+'_'+str(lumi[year_index])+'pb_weighted.root'
# fpath_bkg['ttH'] = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17//MC_Fall18/v1/v90//normalized/ttH_HToSS_SToBB_ms40_pl1000_137000pb_weighted.root'
# fpath_bkg['ggZH'] = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17//MC_Fall18/v2/v90//normalized/ggZHToSS_SToBB_ms40_pl1000_137000pb_weighted.root'
# fpath_bkg['ZH'] = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17//MC_Fall18/v1/v90//normalized/ZHToSS_SToBB_ms40_pl1000_137000pb_weighted.root'

# h_reweight= {}
# v = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/HiggsPtWeights/ttHToggZH_HiggsPtReweight.root'
# root_dir = uproot.open(v) 
# h_reweight['ttH'] = root_dir['higgsPt']
# v = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/HiggsPtWeights/ZHToggZH_HiggsPtReweight.root'
# root_dir = uproot.open(v) 
# h_reweight['ZH'] = root_dir['higgsPt']
NEvents = {}
Total = {}
accep = {}
accep_met = {}
for k,v in fpath_bkg.items():
    print (k, v)
    root_dir = uproot.open(v) 
    tree_bkg[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    if not isData:
#         Total[k] = root_dir['Total'][1]
        Total[k] = root_dir['Total'][1]


        accep[k] = root_dir['acceptance'][1]
        accep_met[k] = root_dir['acceptance_met'][1]
#     else:
#         Nmet200 = root_dir['Nmet200'][1]
#         NmetFilter = root_dir['NmetFilter'][1]
#         Nlep0 = root_dir['Nlep0'][1]
#         Njet1 = root_dir['Njet1'][1]
#         NcosmicVeto = root_dir['NcosmicVeto'][1]
    a = tree_bkg[k]["weight"].array()
    print("NEvents",NEvents[k],len(a))


data /storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17//Data2018//v5/v138//normalized/Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root
NEvents 22763250.0 1671994


# load bdt

In [4]:
# make predictions for test data

import pickle
# model = pickle.load(open( 'bdt_flatten_noEvtSelection.pickle.dat', "rb" ))
bdt_name = 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_XYspread_v12'
bdt_name = 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_XYspread_v12'
# bdt_name = 'bdt_flatten_metfilter_me1112nohits_eta2p0_testsize0p2_noSpread_v2'

# model = pickle.load(open( bdt_name+'.pickle', "rb" ))
# y_pred = model.predict_proba([1,2,3,4,4,5,6])[:, 1]
# print (y_pred)
# BDT_CUT = 0.50882477 

# background cut flow

In [6]:
JET_PT_CUT = 10.0
MUON_PT_CUT = 20.0
N_RECHIT_CUT = 90
jetPt_cut = 50
tightid = False
cut_based = True
cut_based_version="v4"

intime = True
DPHI_CUT = 1
cscRechitClusterNStation10 = {}
cscRechitClusterAvgStation10 = {}
cscRechitClusterEta = {}
sel_ev = {}
weight = {}
dphiMet_cluster = {}
bdt_sel = {}
Nrechits = {}
ncluster = 1
cluster_index = ''
table = OrderedDict()

table['met'] = "Trigger and MET cut "
table['met_filter'] = "MET filters "
table['lep'] = "$N_{lepton} = 0$ "
table['jet'] = r"$N_{jet} \geq 1$ "
table['cosmic_veto'] = r"$N_{\text{CSC+DT rings}} \leq 10 $ "
table['1_cluster'] = "$N_{cluster} \geq 1$ "

table['muon_veto'] = "muon veto "
table['jet_veto'] = "jet veto "
table['time'] = "Time cut "
table['me11'] = "ME1/1 veto "
table['me12'] = "ME1/2 veto "
table['re12'] = "RE1/2 veto "
table['mb1'] = "MB1 veto "
table['rb1'] = "RB1 veto "
table['eta'] = r"$\eta$ cut "
table['time_spread'] = "time spread cut "
table['1_cluster_req'] = "$N_{cluster} = 1$ "



if cut_based: table['bdt'] = "cut-based ID "
else: table['bdt'] = "BDT cut "
table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
table['nrechits'] = r"$N_{rechits}$ cut "

for k, T in tree_bkg.items():
#     if not k == 'data':continue
    weight = T.array('weight')*0.0+1.0
    print(weight[:10], len(weight))
    ########### SELECTION: JETS ############

    sel_jet = np.logical_and(T.array('jetPt') >= jetPt_cut, np.abs(T.array('jetEta')) < 2.4 )
    sel_jet = np.logical_and(sel_jet, T.array('jetTightPassId'))

    ########### SELECTION: EVENTS ############
    NEvents[k] = len(weight)
    sel_ev[k] = np.logical_and(T.array('metEENoise')>=200 ,T.array('METNoMuTrigger'))
    print(len(sel_ev[k]),np.count_nonzero(sel_ev[k]))
    print("trigger + MET", np.count_nonzero(sel_ev[k])/NEvents[k], np.count_nonzero(sel_ev[k])/NEvents[k])
    table["met"] += " & {0:6.3f} & {1:6.3f} & {2:6.0f}".format(100* np.count_nonzero(sel_ev[k])/NEvents[k], 100* np.count_nonzero(sel_ev[k])/NEvents[k], np.count_nonzero(sel_ev[k]))



    acc_met = np.count_nonzero(sel_ev[k])
    sel_temp = np.count_nonzero(sel_ev[k])


    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('Flag2_all') == 1)
    print("MET filters", np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["met_filter"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero(sel_ev[k])/sel_temp, 100*np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k]))




    sel_temp = np.count_nonzero(sel_ev[k])


    sel_ev[k]  = np.logical_and(sel_ev[k], np.logical_and(T.array('category') == 0, T.array('nLeptons') == 0))
    print("0 lepton", np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["lep"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero(sel_ev[k])/sel_temp, 100*np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k]))

    sel_temp = np.count_nonzero(sel_ev[k])


    sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
    print("1 jet", np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["jet"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero(sel_ev[k])/sel_temp, 100*np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k]))

    sel_temp = np.count_nonzero(sel_ev[k])


    sel_ev[k] = np.logical_and(sel_ev[k] , T.array("nDtRings")+T.array("nCscRings") < 10)
    print("cosmic shower veto", np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["cosmic_veto"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero(sel_ev[k])/sel_temp, 100*np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k]))

    sel_temp = np.count_nonzero(sel_ev[k])



    ########### SELECTION: CLUSTERS ############

#     binD = np.logical_and(T.array('cscRechitCluster' + cluster_index + 'Size')>=130, np.abs(T.array('cscRechitCluster' + cluster_index + 'Met_dPhi'))<0.75)
    binD = np.logical_and(T.array('cscRechitCluster' + cluster_index + 'Size')>=130, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi'))<0.75)
    binD = T.array('cscRechitCluster' + cluster_index + 'Size')<0 #for unblinding



    sel_rechitcluster = T.array('cscRechitCluster' + cluster_index + 'Size')>=50
    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_not(binD))
    print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    
    table["1_cluster"] +=  " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met,\
                                                                     np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))  
                  
                          
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])




    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'MuonVetoPt') < MUON_PT_CUT)
#     sel_rechitcluster = np.logical_and( sel_rechitcluster, np.logical_or(T.array('cscRechitCluster' + cluster_index + 'MuonVetoPt') < 10,\
#                               T.array('cscRechitCluster' + cluster_index + 'MuonVetoGlobal') == 0))




    print("muon cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["muon_veto"] +=  " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met,\
                                                                     np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))   
                      
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt') < 10)

    print("jet cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["jet_veto"] +=   " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met,\
                                                                     np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))       
                              
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])
    
    #     sel_rechitcluster = np.logical_and( sel_rechitcluster, np.logical_or(T.array('cscRechitCluster' + cluster_index + 'MuonVetoPt') < MUON_PT_CUT,\
#                               T.array('cscRechitCluster' + cluster_index + 'MuonVetoLooseId') == 0))

 
    
    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeTotal') < 12.5, T.array('cscRechitCluster' + cluster_index + 'TimeTotal') > -5.0))

    print("time cut", np.sum([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["time"] +=  " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met,\
                                                                np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))   
    
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])
    
    me1112_veto = 0
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus11') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus11') <= me1112_veto)

    print("ME11", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["me11"] +=  " & {0:6.2f} & {1:6.3f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met,\
                                                                np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))   
                          
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus12') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus12') <= me1112_veto)

    print("ME12", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["me12"] +=  " & {0:6.2f} & {1:6.3f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met,\
                                                                np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))   
                        
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    


    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_RE12_0p4') <= me1112_veto)

    print("re12 cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["re12"] +=  " & {0:6.2f} & {1:6.3f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met,\
                                                                np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))     
                     
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_MB1Seg_0p4') <= me1112_veto)

    print("mb1 cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["mb1"] +=   " & {0:6.2f} & {1:6.3f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met,\
                                                                np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))       
                                
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])
    
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_RB1_0p4') <= me1112_veto)

    print("rb1 cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["rb1"] +=   " & {0:6.2f} & {1:6.3f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met,\
                                                                np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))   
                      
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])
   

    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'Eta')) < 2.0)

    print("eta cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["eta"] +=   " & {0:6.2f} & {1:6.3f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met,\
                                                                np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))     
                         
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])

    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'TimeSpread') < 20)

    print("time spread cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["time_spread"] +=  " & {0:6.2f} & {1:6.3f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met,\
                                                                       np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))        
                             
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


########### BRANCHES ############

   ##### event variables ##### 

    sel_ev[k] = np.logical_and(sel_ev[k], sel_rechitcluster.sum()==1)
    print("1 cluster", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp)
    table["1_cluster_req"] +=  " & {0:6.2f} & {1:6.3f} & {2:6.0f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp,\
                                                      100*np.sum(weight[sel_ev[k]])/acc_met,\
                                                                       np.sum(weight[sel_ev[k]]))  
    sel_temp = np.sum(weight[sel_ev[k]])


#     ##### bdt variables ####

    cscRechitClusterEta[k] = T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterNStation10[k] = T.array('cscRechitCluster' + cluster_index + 'NStation10')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterAvgStation10[k] = T.array('cscRechitCluster' + cluster_index + 'AvgStation10')[sel_rechitcluster][sel_ev[k]][:,0].flatten()

    if len(cscRechitClusterAvgStation10[k]>0):
        if not cut_based:
            dataset = pd.DataFrame(bdt_var)
            bdt_score[k] = model.predict_proba(dataset.values)[:, 1]
            bdt_sel[k] = bdt_score[k] >= BDT_CUT 
            print("bdt", np.sum(weight[sel_ev[k]][bdt_sel[k]])/acc_met, np.sum(weight[sel_ev[k]][bdt_sel[k]])/sel_temp)
        else:
            if cut_based_version == 'v4':
                cond2 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==2, np.abs(cscRechitClusterEta[k]) < 1.6)
                cond3 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==3, np.abs(cscRechitClusterEta[k]) < 1.6)
                cond4 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==4, np.abs(cscRechitClusterEta[k]) < 1.8)
                cond1 = np.logical_and(cscRechitClusterNStation10[k]==1, np.logical_or(np.logical_or(np.abs(cscRechitClusterAvgStation10[k])==1, cond2), np.logical_or(cond3, cond4)))
                cond2 = np.logical_and(cscRechitClusterNStation10[k] > 1, np.abs(cscRechitClusterEta[k]) < 1.9)
                bdt_sel[k] = np.logical_or(np.logical_or(cond1, cond2), np.logical_or(cond3, cond4))

                print("cut based", np.sum(weight[sel_ev[k]][bdt_sel[k]])/acc_met, np.sum(weight[sel_ev[k]][bdt_sel[k]])/sel_temp)


                bdt_sel[k] = np.logical_or(np.logical_or(cond1, cond2), np.logical_or(cond3, cond4))
            else:print("CUTBASED ERROR")
        dphiMet_cluster[k] = np.abs(T.array('cscRechitCluster' + cluster_index + 'Met_dPhi'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()
        Nrechits[k] = np.abs(T.array('cscRechitCluster' + cluster_index + 'Size'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()
        binD = np.logical_and(dphiMet_cluster[k]<0.75, Nrechits[k]>=130)

        table["bdt"] += " & {0:6.2f} & {1:6.3f}& {2:6.0f}".format(100*np.sum(weight[sel_ev[k]][bdt_sel[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]][bdt_sel[k]])/acc_met,\
                                                        np.sum(weight[sel_ev[k]][bdt_sel[k]]))
        sel_temp =  np.sum(weight[sel_ev[k]][bdt_sel[k]])

#         dphi[k] = np.abs(T.array('jetMet_dPhiMin'))[sel_ev[k]][bdt_sel[k]]




#         bdt_sel[k] = np.logical_and(bdt_sel[k], np.abs(T.array('jetMet_dPhiMin')[sel_ev[k]])<0.6)
#         bdt_sel[k] = np.logical_and(bdt_sel[k], np.abs(T.array('cscRechitCluster' + cluster_index + 'Met_dPhi')[sel_rechitcluster][sel_ev[k]][:,0])<0.75)
        bdt_sel[k] = np.logical_and(bdt_sel[k], np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi')[sel_rechitcluster][sel_ev[k]][:,0])<0.75)
        print("dphi cut", np.sum(weight[sel_ev[k]][bdt_sel[k]])/acc_met, np.sum(weight[sel_ev[k]][bdt_sel[k]])/sel_temp)
        table["dphi"] += " & {0:6.2f} & {1:6.4f}& {2:6.0f}".format(100*np.sum(weight[sel_ev[k]][bdt_sel[k]])/sel_temp, \
                                                         100*np.sum(weight[sel_ev[k]][bdt_sel[k]])/acc_met,  np.sum(weight[sel_ev[k]][bdt_sel[k]]))
        print(T.array('evtNum')[sel_ev[k]][bdt_sel[k]])
        sel_temp =  np.sum(weight[sel_ev[k]][bdt_sel[k]])

        bdt_sel[k] = np.logical_and(bdt_sel[k], np.abs(T.array('cscRechitCluster' + cluster_index + 'Size')[sel_rechitcluster][sel_ev[k]][:,0])>=130)
        print("nrechits cut", np.sum(weight[sel_ev[k]][bdt_sel[k]])/acc_met, np.sum(weight[sel_ev[k]][bdt_sel[k]])/sel_temp)
        table["nrechits"] += " & {0:6.2f} & {1:6.5f}& {2:6.0f}".format(100*np.sum(weight[sel_ev[k]][bdt_sel[k]])/sel_temp, \
                                                                       100*np.sum(weight[sel_ev[k]][bdt_sel[k]])/acc_met, np.sum(weight[sel_ev[k]][bdt_sel[k]]))
        sel_temp =  np.sum(weight[sel_ev[k]][bdt_sel[k]])
        print('signal yield', np.sum(weight[sel_ev[k]][bdt_sel[k]]))

#         bdt_sel[k] = np.logical_and(bdt_sel[k], (T.array('nDTRechitsChamberMinus41')+T.array('nDTRechitsChamberMinus41')+T.array('nDTRechitsChamber40')+\
#         T.array('nDTRechitsChamberPlus41')+T.array('nDTRechitsChamberPlus42'))[sel_ev[k]]<40)
#         print("DTrechits cut", np.sum(weight[sel_ev[k]][bdt_sel[k]])/acc_met, np.sum(weight[sel_ev[k]][bdt_sel[k]])/sel_temp)
#         sel_temp =  np.sum(weight[sel_ev[k]][bdt_sel[k]])



    else:
        table["bdt"] += " & {0:6.2f} & {1:6.2f}".format(0.0, 0.0)
        table["dphi"] += " & {0:6.2f} & {1:6.2f}".format(0.0, 0.0)
        table["nrechits"] += " & {0:6.2f} & {1:6.2f}".format(0.0, 0.0)





for k,v in table.items():
    print(v+r" \\")



[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] 1671994
1671994 1671994
trigger + MET 1.0 1.0
MET filters 0.9442210917024821 0.9442210917024821 1578732
0 lepton 0.9442210917024821 1.0 1578732
1 jet 0.8885779494424023 0.941069795253406 1485697
cosmic shower veto 0.888549241205411 0.9999676919318004 1485649
cluster efficiency 0.5261197109559006 0.5921109225665012
muon cut 0.31229836949175654 0.59358805
jet cut 0.149038214251965 0.4772302
time cut 0.03946246218586909 0.26478082
ME11 0.004720710720253781 0.119625345
ME12 0.002162687186676507 0.45812747
re12 cut 0.0021471368916395634 0.9928097
mb1 cut 0.0021064668892352484 0.9810585
rb1 cut 0.0020998879182580798 0.9968768
eta cut 0.00045693943877789033 0.21760182
time spread cut 0.00033732178464755257 0.7382199
1 cluster 0.00033732178464755257 1.0
cut based 9.928265292818037e-05 0.29432625
dphi cut 3.0502501803236137e-05 0.30722892
[ 415339476 2371873692 2835605820  391981625  476877272  302360679
  979939117  276079011  965787114  447922221  225725803  12

149

# signal cut flow

In [4]:
JET_PT_CUT = 10.0
MUON_PT_CUT = 20.0
N_RECHIT_CUT = 90
jetPt_cut = 50
tightid = False
cut_based = True
cut_based_version="v4"

# BDT_CUT = 0.467 #90% wp vBDT/v1, bdt_flatten_noEvtSelection
# bdtBkgEff =  0.0661076084960519

# BDT_CUT = 0.9227871 #90% wp vBDT/v2, bdt_flatten_metfilter_me1112nohits_eta2p1
# bdtBkgEff =  0.10278372591006424
if bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs':
    BDT_CUT = 0.92245656 #bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs
    bdtBkgEff = 0.09957173447537473
elif bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi':
    BDT_CUT = 0.9608465 #bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi
    bdtBkgEff = 0.06745182012847965
elif bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi_v2':
    BDT_CUT = 0.7802247 #bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi
    bdtBkgEff = 0.061027837259100645
elif bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi_v5':
    BDT_CUT = 0.76344216 #bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi
    bdtBkgEff = 0.051208285385500575
elif bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_XYspread_v12':
    BDT_CUT = 0.50882477 
    bdtBkgEff = 0.18136439267886856
    BDT_BINS = [0.0, BDT_CUT]
    BDT_BINS = [0.0, 0.1]
#     BDT_BINS = [0.0, 0.1, 0.2, BDT_CUT]
    
else:
    print('BDT NAME NOT FOUND')

intime = True
DPHI_CUT = 1
weight = {}
weight_event = {}
weight_ctau = {}
lumiSec = {}
evtNum = {}
nCsc = {}
npv = {}
npu = {}
runNum = {}
nCsc_JetMuonVetoCluster0p4_Me1112Veto = {}
gLLP_csc = {}
cscClusterSize = {}
cscClusterTime = {}
nCscClusters = {}
selections_cluster = {}
sel_cluster = {}
sel_jetveto = {}
met_trigger = {}
met = {}
gLLP_beta = {}
jetPt = {}
jetPhi = {}
metPhi = {}
angle ={}
nLeptons = {}
genJetPt = {}
genJetPhi = {}
genMet = {}
genMetPhi = {}
pileupWeight = {}
gLLP_ctau = {}
npv = {}
nRechitClusters = {}
nJets = {}
nJets_50gev = {}
cscRechitClusterTimeDiff = {}
cscRechitCluster_match_gLLP = {}
cscRechitClusterXYSpread = {}
cscRechitClusterXSpread = {}
cscRechitClusterYSpread = {}
cscRechitClusterNStation = {}
cscRechitClusterEtaPhiSpread = {}
cscRechitClusterPhiSpread = {}
cscRechitClusterEtaSpread = {}
cscRechitClusterNStation10 = {}
cscRechitClusterAvgStation10 = {}
cscRechitClusterX = {}
cscRechitClusterY = {}
cscRechitClusterZ = {}
cscRechitClusterPhi = {}
cscClusterJetVetoPt = {}
cscRechitClusterEta = {}
cscRechitCluster_match_gLLP_index = {}
cscRechitClusterMaxStationRatio = {}
cscRechitClusterNStation = {}
cscRechitClusterNChamber = {}
cscRechitClusterAvgStation = {}
cscRechitClusterMet_dPhi = {}
cscRechitCluster2_match_Me1112_0p8 = {}
jetMet_dPhiMin30 = {}
jetMet_dPhiMin = {}
dphiMet_cluster = {}
nRechits_sr = {}
jetMet_dPhiMin30_sr = {}
gLLP_decay_vertex_z = {}
gLLP_decay_vertex_x = {}
gLLP_decay_vertex_y = {}
gLLP_decay_vertex_r = {}
ctau = {}
bdt_score = {}
a = {}
b = {}
c = {}
d = {}
sel_ev = {}
bdt_sel = {}
gHiggsPt = {}
gLLP_pt = {}
gLLP2_pt = {}
gLLP2_decay_vertex_r = {}
gLLP2_decay_vertex_z = {}
cscRechitCluster2_match_gLLP_decay_z = {}
cscRechitCluster2_match_gLLP_decay_r = {}
gParticleId = {}
gParticlePt = {}
cscRechitCluster_match_gLLP_decay_r = {}
cscRechitCluster_match_gLLP_decay_z = {}
trigger_bits = {}
# keys = ['mc_sr']
dphi = {}
ncluster = 1
cluster_index = '3'
table = OrderedDict()
table['acc'] = "Acceptance "
table['met'] = "Trigger and MET cut "
table['met_filter'] = "MET filters "



table['lep'] = "$N_{lepton} = 0$ "
table['jet'] = r"$N_{jet} \geq 1$ "
table['cosmic_veto'] = r"$N_{\text{CSC+DT rings}} \leq 10 $ "
table['1_cluster'] = "$N_{cluster} \geq 1$ "

table['muon_veto'] = "muon veto "
table['jet_veto'] = "jet veto "
table['time'] = "Time cut "
table['me11'] = "ME1/1 veto "
table['me12'] = "ME1/2 veto "
table['re12'] = "RE1/2 veto "
table['mb1'] = "MB1 veto "
table['rb1'] = "RB1 veto "
table['eta'] = r"$\eta$ cut "
table['time_spread'] = "time spread cut "
table['1_cluster_req'] = "$N_{cluster} = 1$ "

if cut_based: table['bdt'] = "cut-based ID "
else: table['bdt'] = "BDT cut "
table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
table['nrechits'] = r"$N_{rechits}$ cut "

for k, T in tree_bkg.items():
    if k == 'm55ctau100':continue
    weight = T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF')
    print(weight[:10])

#     if not k == 'ggZH':
#         weight = weight*h_reweight[k].values[np.argmax(h_reweight[k].edges>T.array('gHiggsPt')[:,None],axis=1)-1]
#     print(weight[:10])
    ########### SELECTION: JETS ############

    sel_jet = np.logical_and(T.array('jetPt') >= jetPt_cut, np.abs(T.array('jetEta')) < 2.4 )

    ########### SELECTION: EVENTS ############
    hlt = T['HLTDecision'].array()

    sel_ev[k] = np.sum(T.array('gLLP_csc'),axis = 1) >= 1
    acceptance = np.sum((T.array('weight')*T.array('higgsPtWeight'))[sel_ev[k]])
#     total = np.sum(T.array('weight'))
    
#     acceptance = accep[k]
#     total = np.sum(weight_nomet)
    total = np.sum(T.array('weight'))
#     total = Total[k]
#     total = NEvents[k]
    print("acceptance", acceptance/total, np.count_nonzero(sel_ev[k]))
#     table['acc'] = table['acc']+str(acceptance/total)+" &"
    table["acc"] += " & {0:6.3f} & {1:6.3f}".format(100*acceptance/total,100*acceptance/total)

    
    
    
    sel_temp = np.sum(T.array('weight')[sel_ev[k]])
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('METNoMuTrigger'))
#     sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('met')>=200)
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise')>=200)

    accep_met[k] = np.sum(weight[sel_ev[k]])
    print("trigger + MET", accep_met[k]/acceptance, np.sum(weight[sel_ev[k]])/sel_temp, accep_met[k]/total,np.count_nonzero(sel_ev[k]))
#     table["met"] += " & "+str(np.sum(T.array('weight')[sel_ev[k]])/acceptance)+" & "+str(np.sum(T.array('weight')[sel_ev[k]])/total)
#     table["met"] += " & {0:6.3f} & {1:6.3f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/total)
    table["met"] += " & {0:6.3f} & {1:6.3f}".format(100*accep_met[k]/acceptance, 100*accep_met[k]/total)
#     table["met"] += " & {0:6.3f} & {1:6.3f}".format(100*np.sum(weight[sel_ev[k]])/acceptance, 100*np.sum(weight[sel_ev[k]])/total)






    acc_met = np.sum(weight[sel_ev[k]])
    sel_temp = np.sum(weight[sel_ev[k]])


    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('Flag2_all') == 1)
    print("MET filters", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["met_filter"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)



    sel_temp = np.sum(weight[sel_ev[k]])


    sel_ev[k]  = np.logical_and(sel_ev[k], np.logical_and(T.array('category') == 0, T.array('nLeptons') == 0))
    print("0 lepton", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["lep"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)

    sel_temp = np.sum(weight[sel_ev[k]])


    sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
    print("1 jet", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["jet"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)

    sel_temp = np.sum(weight[sel_ev[k]])


    sel_ev[k] = np.logical_and(sel_ev[k] , T.array("nDtRings")+T.array("nCscRings") < 10)
    print("cosmic shower veto", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["cosmic_veto"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)

    sel_temp = np.sum(weight[sel_ev[k]])



    ########### SELECTION: CLUSTERS ############

    sel_rechitcluster = T.array('cscRechitCluster' + cluster_index + '_match_gLLP_csc')
    print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["1_cluster"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)  

    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


#     sel_rechitcluster = np.logical_and( sel_rechitcluster, np.logical_or(T.array('cscRechitCluster' + cluster_index + 'MuonVetoPt') < MUON_PT_CUT,\
#                               T.array('cscRechitCluster' + cluster_index + 'MuonVetoLooseId') == 0))


    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'MuonVetoPt') < MUON_PT_CUT)
    print("muon cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["muon_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)   
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt') < JET_PT_CUT)
    print("jet cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["jet_veto"] +=   " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)       
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])
    
    
    


    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeTotal') < 12.5, T.array('cscRechitCluster' + cluster_index + 'TimeTotal') > -5.0))
    print("time cut", np.sum([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["time"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)   
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])





    me1112_veto = 0
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus11') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus11') <= me1112_veto)
    print("ME11", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["me11"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)   
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus12') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus12') <= me1112_veto)
    print("ME12", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["me12"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)   
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_RE12_0p4') <= me1112_veto)
    print("re12 cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["re12"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)     
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_MB1Seg_0p4') <= me1112_veto)
    print("mb1 cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["mb1"] +=   " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)       
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])
    
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_RB1_0p4') <= me1112_veto)
    print("rb1 cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["rb1"] +=   " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)   
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'Eta')) < 2.0)
    print("eta cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["eta"] +=   " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)     
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'TimeSpread') < 20)
    print("time spread cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["time_spread"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)        
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])




########### BRANCHES ############

   ##### event variables ##### 

#     sel_ev[k] = np.logical_and(sel_ev[k], sel_rechitcluster.sum()==1)
#     print("1 cluster", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp)
#     table["1_cluster_req"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp,\
#                                                       100*np.sum(weight[sel_ev[k]])/acc_met)
#     sel_temp = np.sum(weight[sel_ev[k]])


#     ##### bdt variables ####

    cscRechitClusterEta[k] = T.array('cscRechitCluster' + cluster_index + 'Eta')
    cscRechitClusterNStation10[k] = T.array('cscRechitCluster' + cluster_index + 'NStation10')
    cscRechitClusterAvgStation10[k] = T.array('cscRechitCluster' + cluster_index + 'AvgStation10')
   
    cond2 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==2, np.abs(cscRechitClusterEta[k]) < 1.6)
    cond3 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==3, np.abs(cscRechitClusterEta[k]) < 1.6)
    cond4 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==4, np.abs(cscRechitClusterEta[k]) < 1.8)
    cond1 = np.logical_and(cscRechitClusterNStation10[k]==1, np.logical_or(np.logical_or(np.abs(cscRechitClusterAvgStation10[k])==1, cond2), np.logical_or(cond3, cond4)))
    cond2 = np.logical_and(cscRechitClusterNStation10[k] > 1, np.abs(cscRechitClusterEta[k]) < 1.9)
    bdt_sel[k] = np.logical_or(np.logical_or(cond1, cond2), np.logical_or(cond3, cond4))

    
    sel_rechitcluster = np.logical_and( sel_rechitcluster, bdt_sel[k])
    print("cut based", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["bdt"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)        
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])
    
    sel_rechitcluster = np.logical_and( sel_rechitcluster,np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi'))<0.75)
    print("dphi cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["dphi"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)        
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])
    
    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'Size')>=130)
    print("nrechits", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["nrechits"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)        
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



#         bdt_sel[k] = np.logical_and(bdt_sel[k], (T.array('nDTRechitsChamberMinus41')+T.array('nDTRechitsChamberMinus41')+T.array('nDTRechitsChamber40')+\
#         T.array('nDTRechitsChamberPlus41')+T.array('nDTRechitsChamberPlus42'))[sel_ev[k]]<40)
#         print("DTrechits cut", np.sum(weight[sel_ev[k]][bdt_sel[k]])/acc_met, np.sum(weight[sel_ev[k]][bdt_sel[k]])/sel_temp)
#         sel_temp =  np.sum(weight[sel_ev[k]][bdt_sel[k]])



# else:
#     table["bdt"] += " & {0:6.2f} & {1:6.2f}".format(0.0, 0.0)
#     table["dphi"] += " & {0:6.2f} & {1:6.2f}".format(0.0, 0.0)
#     table["nrechits"] += " & {0:6.2f} & {1:6.2f}".format(0.0, 0.0)

for k,v in table.items():
    print(v+r" \\")


NameError: name 'bdt_name' is not defined